## Задание 1

Опеределите части речи слов в этом тексте, используя пайморфи. В форму запишите слова, которым приписан тэг VERB. 

In [ ]:
# pip install pymorphy2

In [3]:
import pymorphy2 as pm

In [4]:
morph = pm.MorphAnalyzer()

In [5]:
text = "Зонды будут запущены в космос в 2029 году и отправятся ко второй точке Лагранжа в системе Солнце"

In [6]:
for word in text.split():
    p = morph.parse(word)[0]
    print(word, p.tag.POS)


Зонды NOUN
будут VERB
запущены PRTS
в PREP
космос NOUN
в PREP
2029 None
году NOUN
и CONJ
отправятся VERB
ко PREP
второй ADJF
точке NOUN
Лагранжа NOUN
в PREP
системе NOUN
Солнце NOUN


In [7]:
будут,отправятся

NameError: ignored

## Задание 2

На данных предложениях обучите TFIDF векторайзер из sklearn (с дефолтными параметрами). Найдите слово с самым высоким коэффициентом tfidf в первом тексте. Вставьте его в форму.

In [8]:
import requests
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

r = requests.get(
    'https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/anna_karenina.txt'
)
sentences = r.text.split("\n")
# работайте с этими предложениями
sentences = [sent for sent in sentences if len(sent) > 10]

In [9]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)
X.shape

(7536, 33241)

In [10]:
doc = 0
feature_index = X[doc,:].nonzero()[1]
tfidf_scores = zip(feature_index, [X[doc, x] for x in feature_index])

In [11]:
concat = [(vectorizer.get_feature_names_out()[i], s) for (i, s) in tfidf_scores]

In [12]:
most_value = []
for word, score in concat:
    if score == max(concat, key=lambda x: x[1])[1]:
        most_value.append(word)

In [13]:
",".join(most_value)

'вырождается,устои,ослабевают,нравы,падают,буржуазного,натиском,уклада,патриархального,рушатся,страницах,приметами,насыщенное,остропроблемное,психологически'

## Задание 3

Токенизируйте текст Анны Карениной с помощью библиотеки раздел, привидите все к нижнему регистру. Рассчитайте статистики биграммов и униграммов. Найдите наиболее вероятное продолжение "красный ...". Вставьте наиболее веротяное следующее слово в форму.

In [ ]:
pip install razdel gensim

In [16]:
import razdel
import gensim
from razdel import tokenize
from collections import Counter

In [17]:
tokenized = []

for sent in sentences:
    tokens = list(tokenize(sent))
    tokenized.append([_.text.lower() for _ in tokens])

In [18]:
words = [tok for tokens in tokenized for tok in tokens]

## униграммы

In [19]:
vocab = Counter(words)

In [20]:
probas_dvach = Counter({word : c/len(words) for word, c in vocab.items()})
probas_dvach.most_common(20)

[(',', 0.11392826631528649),
 ('.', 0.04796918666106775),
 ('и', 0.03740750930336949),
 ('–', 0.033167779209684195),
 ('не', 0.018931089906025107),
 ('что', 0.016683801277131812),
 ('в', 0.016565065666584614),
 ('он', 0.016072747281388917),
 ('на', 0.010413981842139557),
 ('она', 0.00994483138095307),
 ('с', 0.009637856387831049),
 ('я', 0.009301921489697515),
 ('как', 0.007706230723563226),
 ('но', 0.007474551483471134),
 ('его', 0.007465863511967681),
 ('?', 0.006550730513603916),
 ('это', 0.0064377868840590205),
 ('к', 0.005742749163782743),
 ('ее', 0.005227262854577837),
 ('все', 0.0048363041369224305)]

## биграммы

In [21]:

def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [22]:
tokenized_sent = [['<start>'] + text + ['<end>'] for text in tokenized]

In [23]:
unigrams = Counter()
bigrams = Counter()

for sentence in tokenized_sent:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence))

In [24]:
matrix = np.zeros((len(unigrams), len(unigrams)))
id2word = list(unigrams)
word2id = {word:i for i, word in enumerate(id2word)}

for ngram in bigrams:
    word1, word2 = ngram.split()
    matrix[word2id[word1]][word2id[word2]] =  (bigrams[ngram]/unigrams[word1])

In [25]:
WORD = "красный"

chosen = matrix[word2id[WORD]].argmax()
id2word[chosen]

'мешочек'

## Задание 4

Расчитайте ненормализованное растояние Дамерау-Левенштейна и ненормализованное растояние Левенштейна между этими словами (каждое с каждым), найдите пару слов, для которых эти расстояния отличаются. Вставьте эти слова в форму

In [26]:
# https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%94%D0%B0%D0%BC%D0%B5%D1%80%D0%B0%D1%83_%E2%80%94_%D0%9B%D0%B5%D0%B2%D0%B5%D0%BD%D1%88%D1%82%D0%B5%D0%B9%D0%BD%D0%B0

def damerau_levenshtein_distance(s1, s2):
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)
    for i in range(-1,lenstr1+1):
        d[(i,-1)] = i+1
    for j in range(-1,lenstr2+1):
        d[(-1,j)] = j+1
 
    for i in range(lenstr1):
        for j in range(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1
            d[(i,j)] = min(
                           d[(i-1,j)] + 1, # deletion
                           d[(i,j-1)] + 1, # insertion
                           d[(i-1,j-1)] + cost, # substitution
                          )
            if i and j and s1[i]==s2[j-1] and s1[i-1] == s2[j]:
                d[(i,j)] = min (d[(i,j)], d[i-2,j-2] + 1) # transposition
 
    return d[lenstr1-1,lenstr2-1]

In [27]:
# https://ru.wikibooks.org/wiki/%D0%A0%D0%B5%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8_%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D0%BE%D0%B2/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9B%D0%B5%D0%B2%D0%B5%D0%BD%D1%88%D1%82%D0%B5%D0%B9%D0%BD%D0%B0#Python

def levenshtein_distance(a, b):
    "Calculates the Levenshtein distance between a and b."
    n, m = len(a), len(b)
    if n > m:
        # Make sure n <= m, to use O(min(n, m)) space
        a, b = b, a
        n, m = m, n

    current_row = range(n + 1)  # Keep current and previous row, not entire matrix
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if a[j - 1] != b[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

In [28]:
words = ["решение", "ршеение", "ренешик", "рещиние", "ришение"]

In [29]:
dam_lev_matrix = np.zeros((len(words), len(words)))
lev_matrix = np.zeros((len(words), len(words)))

for i in range(len(words)):
    for j in range(len(words)):
        dam_lev_matrix[i][j] = damerau_levenshtein_distance(words[i], words[j])
        lev_matrix[i][j] = levenshtein_distance(words[i], words[j]) 

In [30]:
dam_lev_matrix - lev_matrix

array([[ 0., -1.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [31]:
print(",".join(words[:2]))

решение,ршеение


## Задание 5

Загрузите токенизатор distilbert-base-uncased из huggingface. Токенизируйте предложения из Анны Карениной с помощью предобученного токенизатора и обучите Fastext модель со следующими параметрами: cbow, размер вектора 300, размер окна 5, минимальный размер символьного нграмма - 2, максимальный размер символьного нграмма - 7. Найдите ближайшее слово к слову "каренина", вставьте его в форму.

In [ ]:
pip install transformers

In [35]:
import tensorflow as tf
from transformers import TFAutoModel
from transformers import AutoTokenizer

In [36]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [44]:
tokenized_sents = []

for sent in sentences:
  marked_text = "[CLS] " + text + " [SEP]"
  tokenized_text = tokenizer.tokenize(marked_text)
  tokenized_sents.append(tokenized_text)

In [48]:
ft = gensim.models.FastText(tokenized_sents,
                            sg=0,
                            min_n=2,
                            max_n=7,
                            size=300,
                            window=5)

In [57]:
ft.wv.most_similar('каренина', topn=40)

[('##на', 0.8245480060577393),
 ('к', 0.6315351128578186),
 ('д', 0.5995323657989502),
 ('##ка', 0.5412423610687256),
 ('б', 0.42186257243156433),
 ('[CLS]', 0.4180133640766144),
 ('##н', 0.31705963611602783),
 ('##я', 0.300679475069046),
 ('##г', 0.23589402437210083),
 ('##ы', 0.17099332809448242),
 ('##и', 0.13701803982257843),
 ('##л', 0.09558025002479553),
 ('##ан', 0.0523027665913105),
 ('##а', 0.02778216451406479),
 ('##ч', -0.09542068839073181),
 ('li', -0.11603137850761414),
 ('с', -0.22643297910690308),
 ('net', -0.3043205738067627),
 ('[SEP]', -0.3249933123588562),
 ('##br', -0.34677812457084656),
 ('##т', -0.35265570878982544),
 ('##s', -0.3887203335762024),
 ('.', -0.5694230794906616)]